In [4]:
import xpress as xp
import pandas as pd
import numpy as np
import math

In [5]:
note_high_table = ['VIP High Table','Centre High Table','End High Table']
note_NSWCG = 'Do not seat with College guests'

ppl_id = [None]

model = xp.problem()
model.controls.maxtime = 400 #max search time
df = pd.read_excel("person2.xlsx")

ppl_type_mapping = {"college guest":1,"fellow":2,"bye-fellow":3,"lecturer":4,"honorary fellow":5,"emeritus fellow":6}


fellow_guest_mapping = []

ppl_typeA = [None]
genderA = [None]
noteA = [None]
to_high_table_id  = []
surnameA = [None]
# not_sit_with = [-1] * 111
# not_sit_with[35] = 46 
# not_sit_with[70] = 42
# not_sit_with[76] = 52
ppl_typeB = []
genderB = []
noteB = []
surnameB = []

In [6]:
for index,value in enumerate(df['GenderA']):
    if df['Dinner'][index] == 2: # fellow with guest
        ppl_typeA.append(ppl_type_mapping[df['Type'][index]])
        ppl_typeB.append(ppl_type_mapping["college guest"])

        if value == 'M':
            genderA.append(1)
        elif value == "F":
            genderA.append(-1)
        else: # None
            genderA.append(0)

        if df['GenderB'][index] == "M":
            genderB.append(1)
        elif df['GenderB'][index] == "F":
            genderB.append(-1)
        else: # None
            genderB.append(0)

        if str(df['Notes'][index]) == "nan":
            noteA.append(None)
        else:
            noteA.append(df['Notes'][index])
        noteB.append(None)
        fellow_guest_mapping.append((len(genderA),len(genderB)))
        if str(df['Surname'][index]) == "nan":
            surnameA.append(None)
        else:
            surnameA.append(df['Surname'][index])
        surnameB.append(df['Fellow Guest'][index])
        ppl_id.append(index+1)
    elif df['Dinner'][index] == 1: # fellow
        ppl_typeA.append(ppl_type_mapping[df['Type'][index]])
        if value == 'M':
            genderA.append(1)
        elif value == "F":
            genderA.append(-1)
        else: # None
            genderA.append(0)
        if str(df['Notes'][index]) == "nan":
            noteA.append(None)
        else:
            noteA.append(df['Notes'][index])
        # note.append(df['Notes'][index])
        if str(df['Surname'][index]) == "nan":
            surnameA.append(None)
        else:
            surnameA.append(df['Surname'][index])
        ppl_id.append(index+1)
#not sit with
note = noteA + noteB
surname = surnameA + surnameB
ppl_type = ppl_typeA + ppl_typeB
gender = genderA + genderB
fellow_guest_mapping = [(i-1,j+len(genderA)-1) for i,j in fellow_guest_mapping]
# ppl_id += [i for i in range(1, len(ppl_type))]
ppl_id += [f'{j}({i}*)' for i,j in fellow_guest_mapping]
NOP = len(ppl_type)-1
NOF = sum([i == -1 for i in gender])
NOM = sum([i == 1  for i in gender])
NON = sum([i == 0 for i in gender])

not_sit_with = []
pplNSWCG = []
for index,n in enumerate(note):
    if n == note_NSWCG:
        pplNSWCG.append(index)
for index,name_target in enumerate(note):
    if name_target is None:
        continue
    
    if "Do not seat near" in name_target or "Do not sit with":
        for name_index, name in enumerate(surname):
            if name is not None:
                if name in name_target:
                    not_sit_with.append((index,name_index))

#ppl to high table
for index,n in enumerate(note):
    if n in note_high_table:
        to_high_table_id.append(index)

num_tables = 6
high_table_id = [ 1, 0, 0, 0, 0, 0 ]
table_seats = [ 21, 16, 26, 10, 20, 17 ]
max_seats = sum(table_seats)
num_ppl = len(ppl_type)-1

NoP = 110
NoF = sum([i == 2 for i in ppl_type])
x=np.array([[xp.var(name=f'x{i}_{j}',vartype=xp.binary) for j in range(num_tables)] for i in range(num_ppl+1)],dtype=xp.npvar)

gender_diff = np.array([xp.var(name=f'gender_diff{i}',vartype=xp.integer) for i in range(num_tables)],dtype=xp.npvar)

max_matrix = np.array([xp.var(name=f'max{i}',vartype=xp.integer) for i in range(num_tables)],dtype=xp.npvar)

min_matrix = np.array([xp.var(name=f'min{i}',vartype=xp.integer) for i in range(num_tables)],dtype=xp.npvar)

model.addVariable(x)

model.addVariable(gender_diff)
model.addVariable(max_matrix)
model.addVariable(min_matrix)

for i in range(1,num_ppl+1):
    model.addConstraint(xp.Sum(x[i][j] for j in range(num_tables)) == 1)
for j in range(num_tables):
    model.addConstraint(xp.Sum(x[i][j] for i in range(1,num_ppl+1)) == table_seats[j])

for f,g in fellow_guest_mapping:
    for j in range(num_tables):
        model.addConstraint(x[f][j] - x[g][j] == 0)

for j in range(num_tables):
    model.addConstraint(xp.Sum(x[i][j] for i in range(1,num_ppl+1) if ppl_type[i] == ppl_type_mapping['fellow']) <= NoF/NoP * table_seats[j] +1)
    model.addConstraint(xp.Sum(x[i][j] for i in range(1,num_ppl+1) if ppl_type[i] == ppl_type_mapping['fellow']) >= NoF/NoP * table_seats[j] -1)

for m,n in not_sit_with:
        for j in range(num_tables):
            model.addConstraint(1-x[m][j]>=x[n][j])


# to high table
for i in to_high_table_id:
    j = 0 #high table     
    model.addConstraint(x[i,j] == 1)

for i in pplNSWCG:
    # breakpoint()
    model.addConstraint(xp.Sum(x[i][j] for j in [1,3,4,5]) == 1)

for j in range(num_tables):
    model.addConstraint(
        gender_diff[j]>=xp.Sum(x[i][j] for i in range(1,num_ppl+1) if gender[i] ==1) - xp.Sum(x[i][j] for i in range(1,num_ppl+1) if gender[i] ==-1)
    )
    model.addConstraint(
        gender_diff[j]>= -xp.Sum(x[i][j] for i in range(1,num_ppl+1) if gender[i] ==1) + xp.Sum(x[i][j] for i in range(1,num_ppl+1) if gender[i] ==-1)
    )

#external constraint for gender
for j in range(num_tables):
    model.addConstraint(xp.Sum(x[i][j]  for i in range(1,num_ppl) if gender[i] == 1) <= NOM/NOP * table_seats[j] +1)
   # model.addConstraint(xp.Sum(x[i][j]  for i in range(1,num_ppl) if gender[i] == -1) <= NOF/NOP * table_seats[j] +1)
   # model.addConstraint(xp.Sum(x[i][j]  for i in range(1,num_ppl) if gender[i] == 0) <= NON/NOP * table_seats[j] +1)

#sum_gender_diff = xp.Sum(gender_diff[j]*gender_diff[j] for j in range(num_tables))
#model.setObjective(sum_gender_diff, xp.minimize)
model.setObjective(xp.Sum(gender_diff), xp.minimize)
#model.setObjective(xp.Sum(x[i][j] for i in range(1,num_ppl+1) for j in range(num_tables)), xp.minimize)
model.optimize()
res = model.getSolution(x)
res_gender_diff =model.getSolution(gender_diff)
table_person_mapping = res.tolist()
table_person_mapping = res.tolist()
table_person_mapping = [[int(i) for i in j] for j in table_person_mapping]
res_to_save = {
    "table_person_mapping":table_person_mapping, 
    "gender":gender ,
    "ppl_type":ppl_type,
    "ppl_id":ppl_id,
    "fellow_guest_mapping":fellow_guest_mapping,
    "not_sit_with":not_sit_with,
    "table_seats":table_seats,
    "num_tables":num_tables,
    "surname":surname,
    "note":note
    
}
import json
with open('res.json', 'w') as fp:
    json.dump(res_to_save, fp)

for j in range(num_tables):
    out = ""
    out += f"table {j+1}:\n"
    allocate_id = [i for i in range(num_ppl+1) if res[i,j] == 1]

    out += f"id: {[ppl_id[i] for i in allocate_id]}\n"
    out += f'note:{[note[i] for i in allocate_id]}\n'
    out += f'name:{[surname[i] for i in allocate_id]}\n'

    num_male = sum([1 if gender[i]==1 else 0 for i in allocate_id])
    num_female = sum([1 if gender[i] == -1 else 0 for i in allocate_id])
    num_unknown_gender = sum([1 if gender[i] == 0 else 0 for i in allocate_id])
    gender_str = []
    for i in allocate_id:
        if gender[i] == 1:
            gender_str.append('male')
        elif gender[i] == -1:
            gender_str.append('female')
        else:
            gender_str.append('unknown')
    out+= f'gender:{gender_str}\n'
    out += f"male: {num_male}, female: {num_female}, unknown: {num_unknown_gender}\n"
    ppl_type = ppl_type
    num_fellow = sum(1 if ppl_type[i] == ppl_type_mapping['fellow'] else 0 for i in allocate_id)
    out +=f"fellow: {num_fellow}\n"

    out += f"gender diff: {res_gender_diff[j]}\n"
    print(out)

FICO Xpress v9.2.4, Hyper, solve started 16:10:08, Aug 21, 2024
Heap usage: 747KB (peak 750KB, 421KB system)
Minimizing MILP noname using up to 8 threads and up to 8006MB memory, with these control settings:
MAXTIME = 400
OUTPUTLOG = 1
Original problem has:
       337 rows          684 cols         2519 elements       684 entities
Presolved problem has:
       121 rows          446 cols         1478 elements       446 entities
LP relaxation tightened
Presolve finished in 0 seconds
Heap usage: 874KB (peak 1347KB, 421KB system)

Coefficient range                    original                 solved        
  Coefficients   [min,max] : [ 1.00e+00,  1.00e+00] / [ 5.00e-01,  1.00e+00]
  RHS and bounds [min,max] : [ 1.00e+00,  2.60e+01] / [ 1.00e+00,  2.10e+01]
  Objective      [min,max] : [ 1.00e+00,  1.00e+00] / [ 1.00e+00,  1.00e+00]
Autoscaling applied standard scaling

Symmetric problem: generators: 58, support set: 390
 Number of orbits: 42, largest orbit: 27
 Row orbits: 7, row support: